In [1]:
from importlib import reload
import sys
from imp import reload
import warnings
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

In [2]:
import pandas as pd
import numpy as np
!pip install nltk
import nltk
import nltk as nlp
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import re
from nltk.stem import WordNetLemmatizer

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


[nltk_data] Downloading package stopwords to C:\Users\dell
[nltk_data]     i5\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data = pd.read_csv("q_quora.csv")

In [4]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.size

5256563

In [6]:
data.shape

(404351, 13)

In [7]:
data.isnull().sum()

id                   0
qid1                 0
qid2                 0
question1            1
question2            2
is_duplicate         0
Unnamed: 6      404014
Unnamed: 7      404324
Unnamed: 8      404343
Unnamed: 9      404348
Unnamed: 10     404349
Unnamed: 11     404349
Unnamed: 12     404349
dtype: int64

In [8]:
data = data.loc[:,['question1', 'question2', 'is_duplicate']]

In [9]:
data.dropna(how = 'any', axis=0)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,How can I be a good geologist?,What should I do to be a great geologist?,1
8,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [10]:
data.shape

(404351, 3)

In [11]:
import string
string.punctuation 

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
#     text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [13]:
data['question1'] = data.question1.apply(lambda x: clean_text(str(x)))

In [14]:
data.head()

,question1,question2,is_duplicate
0,what be the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,what be the story of kohinoor kohinoor diamond,What would happen if the Indian government sto...,0
2,how can i increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,why be i mentally very lonely how can i solve it,Find the remainder when [math]23^{24}[/math] i...,0
4,which one dissolve in water quikly sugar salt ...,Which fish would survive in salt water?,0


In [15]:
data['question2'] = data.question2.apply(lambda x: clean_text(str(x)))
data.head()

,question1,question2,is_duplicate
0,what be the step by step guide to invest in sh...,what be the step by step guide to invest in sh...,0
1,what be the story of kohinoor kohinoor diamond,what would happen if the indian government ste...,0
2,how can i increase the speed of my internet co...,how can internet speed be increase by hack thr...,0
3,why be i mentally very lonely how can i solve it,find the remainder when math2324math be divide...,0
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import linear_model
import numpy as np
from sklearn.model_selection import train_test_split
import scipy
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(pd.concat((data['question1'],data['question2'])).unique())
trainq1_trans = count_vect.transform(data['question1'].values)
trainq2_trans = count_vect.transform(data['question2'].values)
labels = data['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.20, random_state = 42)

In [18]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train,y_train)

print("accuracy: ",rf.score(X_test,y_test))

C:\Users\dell i5\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


accuracy:  0.796156842378603


In [19]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.7508624846978521